In [ ]:
import re

def extraer_primer_numero(cadena):
    # Buscar el primer número en la cadena
    coincidencia = re.search(r'\d+', cadena)
    if coincidencia:
        return int(coincidencia.group())
    else:
        return None

def funcion_de_recompensa(cadena):
    # Diccionario para convertir números en texto a números en entero
    texto_a_numero = {
        "uno": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9
    }

    numero_inicial = extraer_primer_numero(cadena)

    # Extraer los números en texto y sumarlos
    palabras = cadena.split()  # Ignorar "es igual a"
    suma_numeros = sum(texto_a_numero[palabra] for palabra in palabras if palabra in texto_a_numero)

    # Calcular la diferencia
    diferencia = numero_inicial - suma_numeros

    return (9-abs(diferencia))

    if diferencia == 0:
        return 1
    else:
        return 10-abs(diferencia)

funcion_de_recompensa("[INST] 10 es igual a [/INST] nueve más dos")

8

In [ ]:
!pip install transformers
!pip install trl==0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
# version 0.11

# Cargar el tokenizador y el modelo preentrenado
# modelo_nombre = "C:\\Users\\NLP\\Propio\\Research\\Notebooks\\NUMBERS"
modelo_nombre = "gpt2"

# tokenizador = AutoTokenizer.from_pretrained("unsloth/mistral-7b-instruct-v0.2-bnb-4bit")
tokenizador = AutoTokenizer.from_pretrained("gpt2")
modelo = AutoModelForCausalLMWithValueHead.from_pretrained(modelo_nombre)

tokenizador.pad_token = tokenizador.eos_token

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo = modelo.to(device)


# Configurar los parámetros de PPO
config_ppo = PPOConfig(
    batch_size=1,
    forward_batch_size=1,
    ppo_epochs=5,
    learning_rate=1.41e-6
)

# Inicializar el entrenador PPO
entrenador_ppo = PPOTrainer(
    config=config_ppo,
    model=modelo,
    tokenizer=tokenizador
)

# Lista de prompts
# prompts = [
#     "[INST] 10 es igual a [/INST]",
#     "[INST] 13 es igual a [/INST]",
#     "[INST] 16 es igual a [/INST]",
#     "[INST] 19 es igual a [/INST]",
#     "[INST] 22 es igual a [/INST]",
#     "[INST] 25 es igual a [/INST]",
#     "[INST] 28 es igual a [/INST]",
#     "[INST] 31 es igual a [/INST]",
#     "[INST] 34 es igual a [/INST]",
#     "[INST] 37 es igual a [/INST]",
#     "[INST] 40 es igual a [/INST]",
#     "[INST] 43 es igual a [/INST]",
#     "[INST] 46 es igual a [/INST]",
#     "[INST] 49 es igual a [/INST]"
# ]

for epoch in range(60):
    for prompt in prompts:
        # Tokenizar el prompt y convertirlo en tensor
        entradas = tokenizador(prompt, return_tensors='pt').to(device)

        # Generar una respuesta inicial del modelo
        salida = modelo.generate(**entradas)

        respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)

        # Calcular la recompensa utilizando la función personalizada
        recompensa = funcion_de_recompensa2(respuesta)


        print(f"Prompt: {prompt}")
        print(f"Respuesta: {respuesta}")
        print(f"Recompensa: {recompensa}")

        # Preparar los datos para el paso de entrenamiento PPO
        # Convertir prompt y respuesta a listas de tensores
        prompt_tensor = entradas['input_ids'][0]
        respuesta_tensor = salida[0]

        # Asegurarse de que las recompensas estén en una lista de tensores
        recompensa_tensor = torch.tensor([recompensa], dtype=torch.float).to(device)

        # Aplicar un paso de entrenamiento PPO
        entrenador_ppo.step([prompt_tensor], [respuesta_tensor], [recompensa_tensor])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 1+2= [/INST]
Respuesta: [INST] 1+2= [/INST] 2+3= [/INST] 4+5
Recompensa: 0.13613217432458047


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 2+2= [/INST]
Respuesta: [INST] 2+2= [/INST] 2+2+2= [/INST] 2
Recompensa: 0.18962085927556682


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -15.10 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 2+2= [/INST]
Respuesta: [INST] 2+2= [/INST][INST][P][[P][P][
Recompensa: 4.700480365792417


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1313: UserWarning: KL divergence is starting to become negative: -6.39 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Prompt: [INST] 3+3= [/INST]
Respuesta: [INST] 3+3= [/INST] [M][M][M][M][M
Recompensa: 4.700480365792417


KeyboardInterrupt: 

In [ ]:
prompts = [
    "[INST] 1+2= [/INST]",
    "[INST] 2+2= [/INST]",
    "[INST] 3+3= [/INST]",
    "[INST] 1+1= [/INST]",
    "[INST] 0+1= [/INST]"
]
from math import log

def is_digit(x):
  return x in "1234567890"

def funcion_de_recompensa2(cadena):
    answer = cadena.split('[/INST]', 1)[1].strip()

    # if len(answer) > 8:
    #   return -log(len(answer) * 100)

    has_garbage = any(not (c.isdigit() or  c.isspace()) for c in answer)

    numbers = []
    for char in cadena:
      if is_digit(char):
        numbers.append(int(char))
    # print(numbers)
    has_number = len(numbers) > 2

    accurate = 0
    if has_number:
      a,b,c = numbers[:3]
      diff = abs(a+b-c)
      accurate = 10-diff

    result = 10
    if has_number:
      result += 50
    result += accurate * 5
    if has_garbage:
      result -= 9
    return  log(110) - log(result)

funcion_de_recompensa2("[INST] 1+1= [/INST] 2")

0.0

In [ ]:
log(100)

4.605170185988092

In [ ]:
funcion_de_recompensa2("[INST] 1+1= [/INST]!!!!a!")

6.398594934535208